# Creating a Database using sqlite

In [1]:
import sqlite3
import pandas as pd
import os
import re

In [4]:
# path of file with time ranges
times_file_path = 'E:\Physio_Data_Split_Exercise_done\selected_times.csv'

data = pd.read_csv(times_file_path,header=None)

data.head()

,0
0,subject01_RF_05.csv\t1.1482924107142871\t3.699...
1,subject02_RF_05.csv\t0.6509139384920637\t3.911...
2,subject03_RF_05.csv\t1.0330865575396828\t5.042...
3,subject04_RF_05.csv\t1.492857142857142\t5.1612...
4,subject05_RF_05.csv\t1.4544810267857144\t4.407...


In [6]:
def generate_db(db_name, data, descriptions=None):
    """Generate a new data-base, and fill it with the data provided.
    
    Parameters
    ----------
        db_name : str
                Name of the database
        data : pandas DataFrame
                Content of the database
        descriptions: list of string
            More detailed descriptions of the paradigms
            
    Return
    ------
        None
        
    Notes
    -----
        The generated sample database has the following tables, with the corresponding fields
        subjects:    id / weight_kg / height_m / age / gender / first_name / last_name
        paradigms:   id / full_name / abbreviation / short_description
        exercises:   id / subject_id / paradigm_id / num_rep / date / csv_file
        repetitions: id / exercise_id / start_time / stop_time
        
    """
    list_subject_nums = ['01','02','03','04','05','06','07','08','09','10']
    list_paradigm_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2']
    list_repetitions = ['05','10','15']
    
    # Create the database
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    
    # Create the "subjects"-table, and fill it
    subjects_sql = """
        CREATE TABLE subjects (
        id integer PRIMARY KEY,
        weight_kg real,
        height_m real,
        age integer,
        gender text,
        first_name,
        last_name)"""
    cur.execute(subjects_sql)
    
    subjects_sql = "INSERT INTO subjects (id) VALUES (?)"
    for sub_num in list_subject_nums:
        cur.execute(subjects_sql, (int(sub_num),))
    
    
    # Create the "paradigms"-table, and fill it
    paradigms_sql = """
        CREATE TABLE paradigms (
        id integer PRIMARY KEY,
        full_name text,
        abbreviation text NOT NULL,
        short_description text)"""
    cur.execute(paradigms_sql)

    paradigm_sql = "INSERT INTO paradigms (abbreviation) VALUES (?)"
    for paradigm_abbr in list_paradigm_abbrs:
        cur.execute(paradigm_sql, (paradigm_abbr,))
        
    #paradigm_sql = "INSERT INTO paradigms (short_description) VALUES (?)"
    #for descr in descriptions:
    #    cur.execute(paradigm_sql, (descr,))
        
    
    # Create the "exercise"-table
    exercise_sql = """
        CREATE TABLE exercises (
        id integer PRIMARY KEY,
        subject_id integer NOT NULL,
        paradigm_id integer NOT NULL,
        num_rep integer NOT NULL,
        date text,
        csv_file text NOT NULL
        )"""
    conn.execute(exercise_sql)

    
    # Create the "repetitions"-table
    repetitions_sql = """
        CREATE TABLE repetitions (
        id integer PRIMARY KEY,
        sequence_num integer NOT NULL,
        exercise_id integer NOT NULL,
        start_time text NOT NULL,
        stop_time text NOT NULL
        )"""
    conn.execute(repetitions_sql)
    
    
    paradigm_sql = 'SELECT id FROM paradigms WHERE abbreviation = ?'
    exercise_sql = 'INSERT INTO exercises (subject_id, paradigm_id, num_rep, csv_file) VALUES (?,?,?,?)'
    repetitions_sql = 'INSERT INTO repetitions (sequence_num, exercise_id, start_time, stop_time) VALUES (?,?,?,?)'
    
    
    #number_data_points = np.shape(data.values)[0]
    
    for index, line in data.iterrows():
        # get current file name from data
        current_file_name = line[0].split('\t')[0]
        
        # get subject-id from file name
        subject_id = int(re.split('[t_.]',current_file_name)[1])
        
        # get exercise abbreviation from file name
        ex_abbr = re.split('[t_.]',current_file_name)[2]
        
        # get the corresponding paradigm-id
        cur.execute(paradigm_sql, (ex_abbr,))
        paradigm_id = cur.fetchone()[0]
        
        # get number of repetitions from file name
        number_repetitions = int(re.split('[t_.]',current_file_name)[3])
        
        # generate the entry in "exercises", and get the corresponding id
        cur.execute(exercise_sql, (subject_id, paradigm_id, number_repetitions, current_file_name))
        exercise_id = cur.lastrowid
        
        # generate the entries in "repetitions"
        time_points = line[0].split('\t')[1:]
        for ii in range(len(time_points)-1):
            cur.execute(repetitions_sql, (ii+1, exercise_id, time_points[ii], time_points[ii+1]))
        
    conn.commit()
    conn.close()
    return



In [ ]:
db_name = 'DataBase_Physio.db'

if os.path.exists(db_name):
    os.remove(db_name)

generate_db(db_name, data)

In [22]:
# Connect to an existing database
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# Now that the data are entered, an example of how to extract them
query_sql = """
    SELECT e.subject_id, s.first_name,
        p.abbreviation,
        e.num_rep,
        r.sequence_num,
        r.start_time, r.stop_time,
        e.csv_file
    FROM subjects s
    INNER JOIN exercises e
    ON s.id = e.subject_id
    INNER JOIN paradigms p
    ON p.id = e.paradigm_id
    INNER JOIN repetitions r
    ON e.id = r.exercise_id
    """
df = pd.read_sql_query(query_sql, conn)

conn.close()

df

,subject_id,first_name,abbreviation,num_rep,sequence_num,start_time,stop_time,csv_file
0,1,None,RF,5,1,1.1482924107142871,3.699122023809525,subject01_RF_05.csv
1,1,None,RF,5,2,3.699122023809525,6.49581473214286,subject01_RF_05.csv
2,1,None,RF,5,3,6.49581473214286,9.384706101190478,subject01_RF_05.csv
3,1,None,RF,5,4,9.384706101190478,12.073833705357146,subject01_RF_05.csv
4,1,None,RF,5,5,12.073833705357146,14.809060639880954,subject01_RF_05.csv
5,2,None,RF,5,1,0.6509139384920637,3.911928323412699,subject02_RF_05.csv
6,2,None,RF,5,2,3.911928323412699,7.031159474206351,subject02_RF_05.csv
7,2,None,RF,5,3,7.031159474206351,10.398511284722224,subject02_RF_05.csv
8,2,None,RF,5,4,10.398511284722224,13.588634052579367,subject02_RF_05.csv
9,2,None,RF,5,5,13.588634052579367,16.49519035218254,subject02_RF_05.csv


In [23]:
# Connect to an existing database
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# Now that the data are entered, an example of how to extract them
query_sql = """
    SELECT e.subject_id,
        p.abbreviation,
        e.num_rep,
        r.sequence_num,
        r.start_time, r.stop_time,
        e.csv_file
    FROM subjects s
    INNER JOIN exercises e
    ON s.id = e.subject_id
    INNER JOIN paradigms p
    ON p.id = e.paradigm_id
    INNER JOIN repetitions r
    ON e.id = r.exercise_id
    WHERE r.sequence_num > e.num_rep
    """
df2 = pd.read_sql_query(query_sql, conn)

conn.close()

df2

,subject_id,abbreviation,num_rep,sequence_num,start_time,stop_time,csv_file
0,9,LR,10,11,21.704816545758938,23.969116210937507,subject09_LR_10.csv
1,1,BC,10,11,29.502682059151795,32.41607840401787,subject01_BC_10.csv
2,4,BC,10,11,31.739542933872773,35.325371442522325,subject04_BC_10.csv
3,3,BC,15,16,56.598908923921144,60.93714134579615,subject03_BC_15.csv
4,9,TC,10,11,26.7904761904762,29.14761904761906,subject09_TC_10.csv
5,3,TC,15,16,57.79648699079242,61.48594883510046,subject03_TC_15.csv
6,3,TC,15,17,61.48594883510046,65.45491536458334,subject03_TC_15.csv
7,6,MP,10,11,30.631845238095238,33.773065476190474,subject06_MP_10.csv
8,6,MP,10,12,33.773065476190474,37.505208333333336,subject06_MP_10.csv
9,5,MP,15,16,44.84864676339288,47.87193952287949,subject05_MP_15.csv


In [48]:
df3 = df

time_diff = \
    np.array([float(df['stop_time'].values[ii]) for ii in range(len(df['stop_time'].values))]) - \
    np.array([float(df['start_time'].values[ii]) for ii in range(len(df['start_time'].values))])

df3['duration'] = time_diff

df3.describe()

,subject_id,num_rep,sequence_num,duration
count,2710.000000,2710.000000,2710.000000,2710.000000
mean,4.995941,11.666052,6.358672,2.790669
std,2.582915,3.720722,4.001816,0.588107
min,1.000000,5.000000,1.000000,1.140213
25%,3.000000,10.000000,3.000000,2.360409
50%,5.000000,10.000000,6.000000,2.659564
75%,7.000000,15.000000,9.000000,3.121641
max,9.000000,15.000000,17.000000,5.466917
